<a href="https://colab.research.google.com/github/sabbir-mirza-013/My-web-dev-journey/blob/main/Breast_cancer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Loading the dataset**

In [1]:
import os
import json
import base64
import zlib
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import zipfile
from tensorflow.keras import layers, models, losses, metrics, optimizers
from PIL import Image
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:


zip_path = '/content/drive/MyDrive/Breast Cancer Images dataset/breast-ultrasound-images-dataset.zip'
extract_to = '/content/drive/MyDrive/Breast Cancer Images dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Check the contents
os.listdir(extract_to)


['breast-ultrasound-images-dataset.zip', 'breast-ultrasound-images-dataset']

In [4]:

base_path = "/content/drive/MyDrive/Breast Cancer Images dataset/breast-ultrasound-images-dataset"
img_dir = os.path.join(base_path, "img")
ann_dir = os.path.join(base_path, "ann")


In [5]:
print("Sample Images:", os.listdir(img_dir)[:3])
print("Sample Annotations:", os.listdir(ann_dir)[:3])


Sample Images: ['benign (101).png', 'benign (10).png', 'benign (100).png']
Sample Annotations: ['benign (1).png.json', 'benign (100).png.json', 'benign (104).png.json']


**This code defines a custom Keras data generator named BreastCancerDataset for multi-task learning — specifically for breast ultrasound image classification and segmentation. It loads images and their corresponding masks and labels from disk, processes them, and returns them in batches**

In [6]:
# Map label strings to integer class indices
label_map = {"normal": 0, "benign": 1, "malignant": 2}


In [60]:
# class BreastCancerDataset(Sequence):
#     def __init__(self, img_dir, ann_dir, batch_size=8, img_size=256, shuffle=True):
#         self.img_dir = img_dir
#         self.ann_dir = ann_dir
#         self.batch_size = batch_size
#         self.img_size = img_size
#         self.shuffle = shuffle

#         # Match files by name
#         self.filenames = [f for f in os.listdir(self.img_dir) if f.endswith(".png") or f.endswith(".jpg")]
#         self.on_epoch_end()

#     def __len__(self):
#         # Total batches per epoch
#         return int(np.ceil(len(self.filenames) / self.batch_size))

#     def on_epoch_end(self):
#         # Shuffle at epoch end
#         if self.shuffle:
#             np.random.shuffle(self.filenames)

#     def __getitem__(self, index):
#         # Get batch file names
#         batch_files = self.filenames[index * self.batch_size:(index + 1) * self.batch_size]

#         # Allocate batch arrays
#         X = np.zeros((len(batch_files), self.img_size, self.img_size, 1), dtype=np.float32)
#         Y_mask = np.zeros((len(batch_files), self.img_size, self.img_size, 1), dtype=np.float32)
#         Y_label = np.zeros((len(batch_files), 3), dtype=np.float32)  # one-hot

#         for i, file in enumerate(batch_files):
#             img_path = os.path.join(self.img_dir, file)
#             ann_path = os.path.join(self.ann_dir, file.rsplit(".", 1)[0] + ".json")

#             # Load and preprocess image
#             img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#             img = cv2.resize(img, (self.img_size, self.img_size))
#             img = cv2.GaussianBlur(img, (3, 3), 0)
#             img = img.astype(np.float32) / 255.0
#             X[i, ..., 0] = img  # add channel dimension

#             # Load and decode segmentation mask
#             mask = self.decode_mask(ann_path)
#             Y_mask[i, ..., 0] = mask

#             # Extract and one-hot encode label
#             label_str = self.get_label_from_json(ann_path)
#             class_idx = label_map[label_str]
#             Y_label[i, class_idx] = 1.0

#         return X, {"seg_output": Y_mask, "class_output": Y_label}

#     def decode_mask(self, json_path):
#         # Decode the bitmap-based mask
#         with open(json_path, 'r') as f:
#             data = json.load(f)

#         obj = data['objects'][0]
#         origin_x, origin_y = obj['bitmap']['origin']
#         size = data['size']
#         H, W = size['height'], size['width']

#         compressed_data = base64.b64decode(obj['bitmap']['data'])
#         decompressed = zlib.decompress(compressed_data)
#         mask_np = np.frombuffer(decompressed, dtype=np.uint8).reshape((H - origin_y, W - origin_x))

#         full_mask = np.zeros((H, W), dtype=np.uint8)
#         h, w = mask_np.shape
#         full_mask[origin_y:origin_y + h, origin_x:origin_x + w] = mask_np

#         # Resize mask to match image
#         mask_resized = cv2.resize(full_mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
#         return mask_resized

#     def get_label_from_json(self, json_path):
#         with open(json_path, 'r') as f:
#             data = json.load(f)
#         return data['objects'][0]['classTitle']


In [13]:
# import os
# import json
# import base64
# import zlib
# import numpy as np
# import cv2
# import tensorflow as tf
# from tensorflow.keras.utils import Sequence

label_map = {"normal": 0, "benign": 1, "malignant": 2}

class BreastCancerDataset(Sequence):
    def __init__(self, img_dir, ann_dir, batch_size=8, img_size=256, shuffle=True, filenames=None, **kwargs):
        super().__init__(**kwargs)  # Fixes the warning about missing super().__init__

        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle

        # Use custom list (e.g., for train/val split) or full list
        if filenames:
            self.filenames = filenames
        else:
            self.filenames = [f for f in os.listdir(self.img_dir) if f.endswith(('.png', '.jpg'))]

        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.filenames) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.filenames)

    def __getitem__(self, index):
        batch_files = self.filenames[index * self.batch_size:(index + 1) * self.batch_size]

        X = np.zeros((len(batch_files), self.img_size, self.img_size, 1), dtype=np.float32)
        Y_mask = np.zeros((len(batch_files), self.img_size, self.img_size, 1), dtype=np.float32)
        Y_label = np.zeros((len(batch_files), 3), dtype=np.float32)

        for i, file in enumerate(batch_files):
            img_path = os.path.join(self.img_dir, file)
            ann_path = os.path.join(self.ann_dir, file + ".json")  # Fix: .png/.jpg + ".json"

            if not os.path.exists(ann_path):
                print(f"[Warning] Annotation missing for: {file}")
                continue

            # --- Image Preprocessing ---
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = cv2.GaussianBlur(img, (3, 3), 0)
            img = img.astype(np.float32) / 255.0
            X[i, ..., 0] = img

            # --- Mask Decoding ---
            mask = self.decode_mask(ann_path)
            Y_mask[i, ..., 0] = mask

            # --- Label Extraction ---
            label_str = self.get_label_from_json(ann_path)
            class_idx = label_map[label_str]
            Y_label[i, class_idx] = 1.0

        return X, {"seg_output": Y_mask, "class_output": Y_label}



    def decode_mask(self, json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)

        size = data['size']
        H, W = size['height'], size['width']

        # Case: no mask present (e.g., normal case)
        if not data['objects']:
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)

        obj = data['objects'][0]
        origin_x, origin_y = obj['bitmap']['origin']

        # Decode bitmap image from base64+zlib into raw bytes
        compressed_data = base64.b64decode(obj['bitmap']['data'])
        decompressed = zlib.decompress(compressed_data)

        # Decode the raw bytes into a PIL image, then into NumPy array
        mask_img = Image.open(io.BytesIO(decompressed)).convert('L')
        mask_np = np.array(mask_img)

        # Create full-size mask and paste the small mask into it
        full_mask = np.zeros((H, W), dtype=np.uint8)
        h, w = mask_np.shape
        full_mask[origin_y:origin_y + h, origin_x:origin_x + w] = mask_np

        # Resize to match input size (256x256)
        resized_mask = cv2.resize(full_mask, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
        # return resized_mask.astype(np.float32)
        return (resized_mask > 127).astype(np.float32)  # ensure binary



    def get_label_from_json(self, json_path):
        with open(json_path, 'r') as f:
            data = json.load(f)

        if data.get('objects') and 'classTitle' in data['objects'][0]:
            return data['objects'][0]['classTitle']

        # Default label if not found
        return "normal"



**Model Design: Multitask U-Net**

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, metrics, optimizers

IMG_SIZE = 256
NUM_CLASSES = 3

# Dice Loss
# def dice_loss(y_true, y_pred, smooth=1e-6):
#     y_true_f = tf.reshape(y_true, [-1])
#     y_pred_f = tf.reshape(y_pred, [-1])
#     intersection = tf.reduce_sum(y_true_f * y_pred_f)
#     return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred, smooth=1e-6):
    # Ensure predictions are float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Clip predictions to prevent log(0) or NaNs
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0)

    # Flatten per sample
    y_true_flat = tf.reshape(y_true, [tf.shape(y_true)[0], -1])
    y_pred_flat = tf.reshape(y_pred, [tf.shape(y_pred)[0], -1])

    intersection = tf.reduce_sum(y_true_flat * y_pred_flat, axis=1)
    denominator = tf.reduce_sum(y_true_flat + y_pred_flat, axis=1)

    dice = (2. * intersection + smooth) / (denominator + smooth)
    return 1 - tf.reduce_mean(dice)



# U-Net Encoder Block
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    return x

# Build Multitask U-Net Model
def build_multitask_unet(input_shape=(IMG_SIZE, IMG_SIZE, 1)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = conv_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    bn = conv_block(p4, 1024)

    # Segmentation Decoder
    u1 = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(bn)
    u1 = layers.concatenate([u1, c4])
    c5 = conv_block(u1, 512)

    u2 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(c5)
    u2 = layers.concatenate([u2, c3])
    c6 = conv_block(u2, 256)

    u3 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(c6)
    u3 = layers.concatenate([u3, c2])
    c7 = conv_block(u3, 128)

    u4 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(c7)
    u4 = layers.concatenate([u4, c1])
    c8 = conv_block(u4, 64)

    seg_output = layers.Conv2D(1, (1, 1), activation='sigmoid', name='seg_output')(c8)

    # Classification Head
    gap = layers.GlobalAveragePooling2D()(bn)
    dense1 = layers.Dense(128, activation='relu')(gap)
    dropout = layers.Dropout(0.5)(dense1)
    class_output = layers.Dense(NUM_CLASSES, activation='softmax', name='class_output')(dropout)

    model = models.Model(inputs, outputs=[seg_output, class_output])
    return model

# Build model
model = build_multitask_unet()

# Loss functions and weights
losses_dict = {
    'seg_output': dice_loss,
    'class_output': 'categorical_crossentropy'
}
loss_weights = {
    'seg_output': 0.5,
    'class_output': 0.5
}

# Compile model
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=1e-4),
#     loss=losses_dict,
#     loss_weights=loss_weights,
#     metrics={
#         'seg_output': [tf.keras.metrics.MeanIoU(num_classes=2)],
#         'class_output': ['accuracy']
#     }
# )
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss={'seg_output': dice_loss, 'class_output': 'categorical_crossentropy'},
    loss_weights={'seg_output': 0.5, 'class_output': 0.5},
    metrics={'seg_output': [tf.keras.metrics.MeanIoU(num_classes=2)], 'class_output': ['accuracy']}
)


# model.summary()


**Training the Multitask U-Net**

In [15]:
#1. Import and Configure Callbacks

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", verbose=1)
]


In [16]:
#Train/Validation Split

from sklearn.model_selection import train_test_split

all_files = [f for f in os.listdir(img_dir) if f.endswith(".png") or f.endswith(".jpg")]
train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)

train_gen = BreastCancerDataset(img_dir, ann_dir, batch_size=8, img_size=256)
train_gen.filenames = train_files

val_gen = BreastCancerDataset(img_dir, ann_dir, batch_size=8, img_size=256, shuffle=False)
val_gen.filenames = val_files



In [ ]:

# Train the Model

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
18/78 ━━━━━━━━━━━━━━━━━━━━ 56:08 56s/step - class_output_accuracy: 0.5495 - class_output_loss: 1.0712 - loss: 0.9738 - seg_output_loss: 0.8763 - seg_output_mean_io_u: 0.4573